In [1]:
import pandas as pd
import numpy as np
import torch 
import chess.pgn
import os
import fnmatch
import math
from stockfish import Stockfish
from torch.autograd import Variable
from tqdm import tqdm
from torch.utils.data import DataLoader
from stockfish import Stockfish
from torchtext.vocab import build_vocab_from_iterator
from torch.optim import Adam

## Import the data

In [2]:
main_folder = os.path.join(os.getcwd().split("RL_chess_engine")[0], "RL_chess_engine")
games_folder= os.path.join(main_folder, "src", "games")
path_real_games = os.path.join(games_folder, "real_games")

In [3]:
FENS=[]
TARGET_MOVES=[]
LAST_MOVES=[]
stockfish = Stockfish()
for (dirpath, dirnames, filenames) in os.walk(path_real_games):
    for file_path in tqdm(filenames[:100]):
        with open(path_real_games+"/"+file_path) as file:
            game = chess.pgn.read_game(file)
            if game is None:
                continue
            for _ in game.mainline():
                fen=game.board().fen()
                FENS.append(fen)
                stockfish.set_fen_position(fen) 
                TARGET_MOVES.append(stockfish.get_best_move())
                if game.move==None:
                    LAST_MOVES.append('start')
                else:
                    LAST_MOVES.append(game.move.uci())
                game=game.next()
FENS       

100%|█████████████████████████████████████████| 100/100 [46:33<00:00, 27.93s/it]


['rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1',
 'rnbqkbnr/pppppppp/8/8/3P4/8/PPP1PPPP/RNBQKBNR b KQkq - 0 1',
 'rnbqkbnr/ppp1pppp/8/3p4/3P4/8/PPP1PPPP/RNBQKBNR w KQkq - 0 2',
 'rnbqkbnr/ppp1pppp/8/3p4/3P4/2N5/PPP1PPPP/R1BQKBNR b KQkq - 1 2',
 'rnbqkbnr/ppp2ppp/8/3pp3/3P4/2N5/PPP1PPPP/R1BQKBNR w KQkq - 0 3',
 'rnbqkbnr/ppp2ppp/8/3pp3/3PP3/2N5/PPP2PPP/R1BQKBNR b KQkq - 0 3',
 'rnbqkbnr/ppp2ppp/8/3p4/3pP3/2N5/PPP2PPP/R1BQKBNR w KQkq - 0 4',
 'rnbqkbnr/ppp2ppp/8/3p4/3QP3/2N5/PPP2PPP/R1B1KBNR b KQkq - 0 4',
 'rnbqkbnr/ppp2ppp/8/8/3Qp3/2N5/PPP2PPP/R1B1KBNR w KQkq - 0 5',
 'rnbqkbnr/ppp2ppp/8/8/4Q3/2N5/PPP2PPP/R1B1KBNR b KQkq - 0 5',
 'rnbqk1nr/ppp1bppp/8/8/4Q3/2N5/PPP2PPP/R1B1KBNR w KQkq - 1 6',
 'rnbqk1nr/ppp1bppp/8/8/4Q3/2N2N2/PPP2PPP/R1B1KB1R b KQkq - 2 6',
 'rnbqk2r/ppp1bppp/5n2/8/4Q3/2N2N2/PPP2PPP/R1B1KB1R w KQkq - 3 7',
 'rnbqk2r/ppp1bppp/5n2/8/8/2N1QN2/PPP2PPP/R1B1KB1R b KQkq - 4 7',
 'rnbqk2r/pp2bppp/5n2/2p5/8/2N1QN2/PPP2PPP/R1B1KB1R w KQkq - 0 8',
 'rnbqk2r/pp2bppp/5n2

In [108]:
data_pd=pd.DataFrame({'fens':FENS,'moves':LAST_MOVES,'target_moves':TARGET_MOVES,'LEGAL_MOVES':LEGAL_MOVES})

In [122]:
data_pd.to_csv("./data/100_games.csv")

FileNotFoundError: [Errno 2] No such file or directory: './data/100_games.csv'

In [107]:
LEGAL_MOVES=[]
for (dirpath, dirnames, filenames) in os.walk(path_real_games):
    for file_path in tqdm(filenames[:100]):
        with open(path_real_games+"/"+file_path) as file:
            game = chess.pgn.read_game(file)
            if game is None:
                continue
            for _ in game.mainline():
                fen=game.board().fen()
                LEGAL_MOVES.append([move.uci() for move in game.board().legal_moves])
                game=game.next()
max_length=66
LEGAL_MOVES=[moves+['end']*(max_length-len(moves)) for moves in LEGAL_MOVES]

100%|█████████████████████████████████████████| 100/100 [00:05<00:00, 17.03it/s]


## Let's get all possible moves

In [7]:
letters=list('abcdefgh')
pawn_crown=[]
pieces=list('qnrb')
for x in range(len(letters)):
    for piece in pieces:
        if letters[x]== 'a':
            pawn_crown.append(letters[x]+'2'+letters[x]+'1'+piece)
            pawn_crown.append(letters[x]+'2'+letters[x+1]+'1'+piece)
            pawn_crown.append(letters[x]+'7'+letters[x]+'8'+piece)
            pawn_crown.append(letters[x]+'7'+letters[x+1]+'8'+piece)
        elif letters[x]== 'h':

            pawn_crown.append(letters[x]+'2'+letters[x]+'1'+piece)
            pawn_crown.append(letters[x]+'2'+letters[x-1]+'1'+piece)
            pawn_crown.append(letters[x]+'7'+letters[x]+'8'+piece)
            pawn_crown.append(letters[x]+'7'+letters[x-1]+'8'+piece)

        else:
            pawn_crown.append(letters[x]+'2'+letters[x]+'1'+piece)
            pawn_crown.append(letters[x]+'2'+letters[x-1]+'1'+piece)
            pawn_crown.append(letters[x]+'2'+letters[x+1]+'1'+piece)
            pawn_crown.append(letters[x]+'7'+letters[x]+'8'+piece)
            pawn_crown.append(letters[x]+'7'+letters[x-1]+'8'+piece)
            pawn_crown.append(letters[x]+'7'+letters[x+1]+'8'+piece)
            
board_spaces=[x+str(y) for x in letters for y in range(1,9)]
possible_moves=[x+str(y) for x in board_spaces for y in board_spaces]+pawn_crown
moves_dict={ key:label for label,key in enumerate(possible_moves)}
moves_dict['start']=len(moves_dict.values())
moves_dict['end']=len(moves_dict.values())
inv_moves_dict = {v: k for k, v in moves_dict.items()}

## Let's tokenize the FENs

In [28]:
class Tokenizer:
    def __init__(self,path=None, max_length=76,vocab_size=323):
        self.max_length=max_length
        self.dictionary_board= {
            "/":0, #change value, and keep zero to mask
            "r":9,
            "n":10,
            "b":11,
            "q":12,
            "k":13,
            "p":14,
            "R":15,
            "N":16,
            "B":17,
            "Q":18,
            "K":19,
            "P":20,
            " ":321,
            "<PAD>":322
        }
        self.detokenizer_dict={v: k for k, v in self.dictionary_board.items()}
        self.dictionary_active_color={
            "w":21,
            "b":22
        }
        self.detokenizer_dict.update({v: k for k, v in self.dictionary_active_color.items()})
        self.dictionary_castling_rights={
            "K":23,
            "Q":24,
            "k":25,
            "q":26,
            "-":27
        }
        self.detokenizer_dict.update({v: k for k, v in self.dictionary_castling_rights.items()})
        self.dictionary_en_passant_square={
            "-":28,
            "a3":29,
            "b3":30,
            "c3":31,
            "d3":32,
            "e3":33,
            "f3":34,
            "g3":35,
            "h3":36,
            "a6":37,
            "b6":38,
            "c6":39,
            "d6":40,
            "e6":41,
            "f6":42,
            "g6":43,
            "h6":44
        }
        self.detokenizer_dict.update({v: k for k, v in self.dictionary_en_passant_square.items()})
        self.dictionary_half_move_clock={
        }
        self.dictionary_full_move_number={}
        
        self.vocab_size=vocab_size
        
        
    def board_to_token_vector(self, board):
        #print(board)
        #takes every charater on the board and returns a token vector
        return [ int(char) if char.isnumeric() else self.dictionary_board[char] for char in board ]



    def tokenize(self, fen):
        token_vector =[]
        fen_components=fen.split(" ")

        token_vector+= self.board_to_token_vector(fen_components[0])
        token_vector+=[self.dictionary_board[" "]]
        token_vector+= [self.dictionary_active_color[fen_components[1]]] # tokenize the active color
        token_vector+=[self.dictionary_board[" "]]
        token_vector+= [self.dictionary_castling_rights[char] for char in fen_components[2] ] # tokenize the castling rights
        token_vector+=[self.dictionary_board[" "]]
        token_vector+= [self.dictionary_en_passant_square[fen_components[3]]] # tokenize the en passant square
        token_vector+=[self.dictionary_board[" "]]
        token_vector+= [int(fen_components[4])+ 45] # tokenize the half move clock
        token_vector+=[self.dictionary_board[" "]]
        token_vector+= [int(fen_components[5])+ 45+100] # tokenize the full move number
        if(len(token_vector)<self.max_length):
            token_vector+=[self.dictionary_board["<PAD>"]]*(self.max_length-len(token_vector))
        else:
            print("bigger:", len(token_vector))
            
    
        return torch.LongTensor(token_vector)
    def detokenize(self,token_vector):
        fen=[]
        for token in token_vector:
            token=token.item()
            if token in range(1,9):
                fen.append(str(token))
            elif token > 44 and token<45+100:
                fen.append(str(token- 45))
            elif token >= 45+100 and token<self.dictionary_board[" "]:
                fen.append(str(token-(45+100)))
            else:
                fen.append(self.detokenizer_dict[token])
        return ''.join(fen).replace("<PAD>", "")
        

## Let's prepare the dataset

In [111]:
tokenizer=Tokenizer()
class Dataset(torch.utils.data.Dataset):

    def __init__(self, fens,legal_moves,target_moves,moves_dict):
        self.labels_dict=moves_dict
        self.labels = [self.labels_dict[label] for label in target_moves]
        self.legal_moves=[[self.labels_dict[move] for move in list_moves] for list_moves in legal_moves]
        self.positions = [tokenizer.tokenize(fen) for fen in fens]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_positions(self, idx):
        # Fetch a batch of inputs
        return self.positions[idx]
    def get_batch_legal_moves(self, idx):
        # Fetch a batch of inputs
        return self.legal_moves[idx]

    def __getitem__(self, idx):

        batch_positions = self.get_batch_positions(idx)
        batch_y = self.get_batch_labels(idx)
        batch_input_tgt=torch.LongTensor(self.get_batch_legal_moves(idx))

        return batch_positions, batch_input_tgt,batch_y

In [118]:
class PositionalEncoding(torch.nn.Module):
    "Implement the PE function."
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = torch.nn.Dropout(p=dropout)
        
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) *
                             -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + Variable(self.pe[:x.size(0),:], 
                         requires_grad=False)
        return self.dropout(x)

class TransformerClassifier(torch.nn.Module):

    def __init__(self, vocab_size_fens,n_moves, dropout=0.1, d_model=512, n_labels=5, nhead=8, num_encoder_layers=4, dim_feedforward=2048):

        super(TransformerClassifier, self).__init__()
        self.d_model=d_model
        self.embedding_fens = torch.nn.EmbeddingBag(vocab_size_fens, d_model)
        self.embedding_moves = torch.nn.EmbeddingBag(n_moves, d_model)
        self.pos_encoder_fens = PositionalEncoding(d_model, dropout)
        self.pos_encoder_moves = PositionalEncoding(d_model, dropout)
        self.transformer_model = torch.nn.Transformer(nhead=nhead, num_encoder_layers=num_encoder_layers
                                                ,num_decoder_layers=num_encoder_layers,dim_feedforward=dim_feedforward,
                                               d_model=d_model)
        self.dropout = torch.nn.Dropout(dropout)
        self.linear = torch.nn.Linear(d_model, n_labels)
        self.relu = torch.nn.ReLU()

    def forward(self, fens_vector, mask_fens, moves_vector,mask_moves):
        embeded_fens=self.embedding_fens(fens_vector)* np.sqrt(self.d_model)
        fens_encoded=self.pos_encoder_fens(embeded_fens)
        embeded_moves=self.embedding_moves(moves_vector)* np.sqrt(self.d_model)
        moves_encoded=self.pos_encoder_moves(embeded_moves)
        
        
        pooled_output = self.transformer_model(src=fens_encoded,tgt=moves_encoded)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer
    
transformer=TransformerClassifier(vocab_size_fens=tokenizer.vocab_size,n_moves=len(moves_dict.keys()),
                                  d_model=(len(moves_dict.keys()) -2), 
                                  n_labels=len(moves_dict.keys()), dim_feedforward=(len(moves_dict.keys())-2))


In [119]:
EPOCHS = 60
LR = 1e-4
MOMENTUM = 0.9
fens_train, fens_val = np.split(FENS, [int(.8*len(FENS))])
last_moves_train, last_moves_val = np.split(LEGAL_MOVES, [int(.8*len(LEGAL_MOVES))])
target_moves_train, target_val = np.split(TARGET_MOVES, [int(.8*len(TARGET_MOVES))])
train, val = Dataset(fens_train,last_moves_train,target_moves_train,
                     moves_dict),Dataset(fens_val,last_moves_val,target_val,moves_dict)

BATCH_SIZE=200
train_dataloader = torch.utils.data.DataLoader(train, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val, batch_size=BATCH_SIZE)

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(transformer.parameters(), lr= LR, momentum=MOMENTUM)

for epoch_num in range(EPOCHS):
    total_acc_train = 0
    total_loss_train = 0
    
    for train_input, target_input, train_label, in tqdm(train_dataloader):
        
        train_label = train_label.to(device)
        target_input =target_input.to(device)
        #target_input =torch.reshape(target_input,(target_input.size()[0],1))
        mask_target = target_input.to(device)
        mask_input = train_input.to(device)
        input_id = train_input.squeeze(1).to(device)
        
        output=transformer(input_id , mask_input,target_input,mask_target)
        batch_loss = criterion(output, train_label)
        total_loss_train += batch_loss.item()
        
        acc = (output.argmax(dim=1) == train_label).sum().item()
        total_acc_train += acc

        transformer.zero_grad()
        batch_loss.backward()
        optimizer.step()
        
    total_acc_val = 0
    total_loss_val = 0
    with torch.no_grad():

        for val_input, target_input,val_label in val_dataloader:

            val_label = val_label.to(device)
            target_input =target_input.to(device)
            #target_input =torch.reshape( target_input,(target_input.size()[0],1))
            mask_target = target_input.to(device)
            mask = val_input.to(device)
            input_id = val_input.squeeze(1).to(device)
            

            output= transformer(input_id , mask_input,target_input,mask_target)
            
            batch_loss = criterion(output, val_label)
            total_loss_val += batch_loss.item()

            acc = (output.argmax(dim=1) == val_label).sum().item()
            total_acc_val += acc

    print(
        f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train): .3f} \
        | Train Accuracy: {total_acc_train / len(train): .3f} \
        | Val Loss: {total_loss_val / len(val): .3f} \
        | Val Accuracy: {total_acc_val / len(val): .3f}')


100%|███████████████████████████████████████████| 25/25 [40:40<00:00, 97.64s/it]


Epochs: 1 | Train Loss:  0.042         | Train Accuracy:  0.008         | Val Loss:  0.045         | Val Accuracy:  0.034


100%|██████████████████████████████████████████| 25/25 [42:40<00:00, 102.44s/it]


Epochs: 2 | Train Loss:  0.040         | Train Accuracy:  0.025         | Val Loss:  0.044         | Val Accuracy:  0.028


100%|███████████████████████████████████████████| 25/25 [40:49<00:00, 98.00s/it]


Epochs: 3 | Train Loss:  0.039         | Train Accuracy:  0.022         | Val Loss:  0.042         | Val Accuracy:  0.028


  8%|███▎                                     | 2/25 [05:19<1:01:08, 159.51s/it]


KeyboardInterrupt: 

In [12]:
torch.save(transformer.state_dict(), '../../models/data_scientist/chess_transformer_v2.pth')

In [117]:
transformer.eval()
train=Dataset(['rnbqkb1r/pp2pppp/5n2/2pp4/3P4/2N5/PPP1PPPP/R1BQKB1R w KQkq - 0 1'],['c7c5'],['c6b5'],moves_dict)
for fen,prev_move, best_move in torch.utils.data.DataLoader(train, batch_size=1):
    output=transformer(fen,fen,torch.reshape(prev_move,(prev_move.size()[0],1)),prev_move)
    _,response_move=torch.max(output,1)
    response_move=response_move.item()
    print(torch.topk(output,15))
    print([ inv_moves_dict[i.item()] for i in torch.topk(output,15).indices[0]])
    print(inv_moves_dict[response_move])

KeyError: 'c'

In [41]:
(len(moves_dict.keys()) -2)//4

1068

In [97]:
train.__getitem__(15)

(tensor([  9,  10,  11,  12,  13,   2,   9,   0,  14,  14,   2,  11,  14,  14,
          14,   0,   5,  10,   2,   0,   1,  17,  14,   5,   0,   8,   0,   2,
          16,   1,  18,  16,   2,   0,  20,  20,  20,   2,  20,  20,  20,   0,
          15,   1,  17,   1,  19,   2,  15, 321,  22, 321,  23,  24,  25,  26,
         321,  28, 321,  46, 321, 153, 322, 322, 322, 322, 322, 322, 322, 322,
         322, 322, 322, 322, 322, 322]),
 tensor([2543, 2014, 1502,  990,  981, 2910]),
 array(1502))

In [110]:
max_length=66
POST_LEGAL_MOVES

[['g1h3',
  'g1f3',
  'b1c3',
  'b1a3',
  'h2h3',
  'g2g3',
  'f2f3',
  'e2e3',
  'd2d3',
  'c2c3',
  'b2b3',
  'a2a3',
  'h2h4',
  'g2g4',
  'f2f4',
  'e2e4',
  'd2d4',
  'c2c4',
  'b2b4',
  'a2a4',
  'end',
  'end',
  'end',
  'end',
  'end',
  'end',
  'end',
  'end',
  'end',
  'end',
  'end',
  'end',
  'end',
  'end',
  'end',
  'end',
  'end',
  'end',
  'end',
  'end',
  'end',
  'end',
  'end',
  'end',
  'end',
  'end',
  'end',
  'end',
  'end',
  'end',
  'end',
  'end',
  'end',
  'end',
  'end',
  'end',
  'end',
  'end',
  'end',
  'end',
  'end',
  'end',
  'end',
  'end',
  'end',
  'end'],
 ['g8h6',
  'g8f6',
  'b8c6',
  'b8a6',
  'h7h6',
  'g7g6',
  'f7f6',
  'e7e6',
  'd7d6',
  'c7c6',
  'b7b6',
  'a7a6',
  'h7h5',
  'g7g5',
  'f7f5',
  'e7e5',
  'd7d5',
  'c7c5',
  'b7b5',
  'a7a5',
  'end',
  'end',
  'end',
  'end',
  'end',
  'end',
  'end',
  'end',
  'end',
  'end',
  'end',
  'end',
  'end',
  'end',
  'end',
  'end',
  'end',
  'end',
  'end',
  'end',
  'en